In [ ]:
#Import data

import phate
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sch
import scipy.cluster.hierarchy as sch

In [ ]:
Data = pd.read_csv('')

In [ ]:
#Run PHATE

import phate

phate_op = phate.PHATE(knn = 3)
Data_phate = phate_op.fit_transform(Data)

phate.plot.scatter2d(Data_phate, figsize=(30,10), s= 700, alpha =1, c = Labels, cmap = ('#FFC0CB', '#A9A9A9'), edgecolors = 'black')

matplotlib.rcParams['figure.figsize'] = (25, 10)
phate.plot.scatter2d(Data_phate, s= 1000, alpha =1, c = Labels, cmap = ('#E26983', '#459E97', '#459E97'), edgecolors = 'black')
plt.ylim((-0.05, 0.1))
plt.xlim((-0.05, 0.11))
plt.grid = False
plt.top = False
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.ylabel(r'PHATE 1', fontsize=47)
plt.xlabel(r'PHATE 2', fontsize=47)
ax = plt.gca()
ax.legend_ = None
plt.savefig('', dpi=300, bbox_inches = 'tight', transparent = True)




In [ ]:
#Run Pseudotime

import anndata
import scanpy

# Here we create an AnnData container
adata = anndata.AnnData(Data)

# DPT requires a root cell, depending on the orientation of your
# plot, you should be able to use the cell with the lowest 
# PHATE1 coordinate

adata.uns['iroot'] = np.argmin(Data_phate[:,0])

# Run diffusion pseudotime

scanpy.pp.pca(adata)
scanpy.pp.neighbors(adata)
scanpy.tl.diffmap(adata)
scanpy.tl.dpt(adata, n_branchings=0)

# Grab the output

dpt = adata.obs['dpt_pseudotime'].to_numpy()

In [ ]:
# plot

scprep.plot.scatter2d(data_phate, c=dpt)

In [ ]:
#Build a knn graph

import graphtools
G = graphtools.Graph(data, knn=5, decay=None, kernel_symm=None)

In [ ]:
#Measure knn group identity by pseudotime bin

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10, 10)

step = 0.2
bin_proportions = []
bin_starts = np.arange(np.min(dpt), np.max(dpt), step)
for bin_start in bin_starts:
    # find which points sit in this bin
    bin_points = np.argwhere((dpt >=  bin_start) & (dpt < bin_start + step)).flatten()
    bin_proportion = []
    for idx in bin_points:
        # find this point's 5 nearest neighbors
        idx_neighbors = G.kernel[idx].astype(bool).toarray().flatten()
        # calculate proportion of neighbors from the same group
        idx_proportion = np.mean(Labels_New[idx_neighbors] == Labels_New[idx])
        bin_proportion.append(idx_proportion)
    # take the mean across all points in the bin
    bin_proportions.append(np.mean(bin_proportion))
    
# plot
import matplotlib.pyplot as plt
plt.plot(bin_starts, bin_proportions)
plt.ylim((0, 1))
plt.xlim((0, 1))
plt.grid = False
plt.top = False
plt.gca().spines['right'].set_color('none')
plt.gca().spines['top'].set_color('none')
plt.ylabel(r'Proportion of Cells', fontsize=20)
plt.xlabel(r'Pseudotime', fontsize=20)
ax = plt.gca()

ax.legend_ = None